### Compare results against a similar code using pytorch:

In [ ]:
import os
import torch
from torch import nn
import torch.optim as optim
import torch.utils.data as data_utils

import numpy as np 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

cancer = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(cancer.data,cancer.target,test_size=0.2,random_state=0)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_test_scaled = torch.from_numpy(X_test_scaled).float()
y_test = torch.from_numpy(y_test).float()

trainset = data_utils.TensorDataset(torch.from_numpy(X_train_scaled).float(),
                                 torch.from_numpy(y_train).float())

trainloader = data_utils.DataLoader(trainset, batch_size=4, shuffle=True)


In [ ]:


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.linear_stack = nn.Sequential(
            nn.Linear(30, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits
    


    
net = NeuralNetwork()
print(net)

In [ ]:

epochs = 50
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.1)#, momentum=0.9)


for epoch in range(epochs): 

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        #unsqueeze to broadcast: 
        labels = labels.unsqueeze(-1)
        # zero gradients to avoid accum.grads
        optimizer.zero_grad()

        # forward -> backward -> optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        #print(loss)
        loss.backward()
        optimizer.step()


### Testing accuracy: 

In [ ]:

def accuracy(X_test, y_test):
    
    y_hat = net.forward(X_test)
    prediction_val = [1 if x > 0.5 else 0 for x in y_hat.data.numpy()]
    correct_val = (prediction_val == y_test.numpy()).sum()
    return correct_val/len(y_test)

accuracy(X_test_scaled, y_test)